In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import utils

In [3]:
#adding cal-itp id 
bbb = 300
st = 290

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
rt_sched = utils.read_data()

In [6]:
rt_sched.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
0,290,SamTrans,1,110,110,2022-06-30,2021-04-16,2099-01-01,48,48,1.000000,Thursday,June
1,290,SamTrans,1,110,110,2022-06-23,2021-04-16,2099-01-01,48,24,0.500000,Thursday,June
2,290,SamTrans,1,110,110,2022-06-24,2021-04-16,2099-01-01,48,28,0.583333,Friday,June
3,290,SamTrans,1,110,110,2022-06-27,2021-04-16,2099-01-01,48,35,0.729167,Monday,June
4,290,SamTrans,1,110,110,2022-06-29,2021-04-16,2099-01-01,48,48,1.000000,Wednesday,June


In [7]:
rt_sched.tail()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
6893,300,Big Blue Bus,0,3479,1,2022-05-12,2022-03-08,2099-01-01,180,180,1.000000,Thursday,May
6894,300,Big Blue Bus,0,3479,1,2022-06-09,2022-03-08,2099-01-01,180,169,0.938889,Thursday,June
6895,300,Big Blue Bus,0,3479,1,2022-06-24,2022-03-08,2099-01-01,180,172,0.955556,Friday,June
6896,300,Big Blue Bus,0,3479,1,2022-06-21,2022-03-08,2099-01-01,180,180,1.000000,Tuesday,June
6897,300,Big Blue Bus,0,3479,1,2022-06-30,2022-03-08,2099-01-01,180,177,0.983333,Thursday,June


In [8]:
rt_sched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6898 entries, 0 to 6897
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   calitp_itp_id        6898 non-null   int64         
 1   agency_name          3950 non-null   object        
 2   calitp_url_number    6898 non-null   int64         
 3   route_id             6898 non-null   object        
 4   route_short_name     6898 non-null   object        
 5   service_date         6898 non-null   datetime64[ns]
 6   calitp_extracted_at  6898 non-null   object        
 7   calitp_deleted_at    6898 non-null   object        
 8   num_sched            6898 non-null   int64         
 9   num_vp               6898 non-null   int64         
 10  pct_w_vp             6898 non-null   float64       
 11  weekday              6898 non-null   object        
 12  month                6898 non-null   object        
dtypes: datetime64[ns](1), float64(1),

In [9]:
rt_sched.agency_name.value_counts()

SamTrans        2948
Big Blue Bus    1002
Name: agency_name, dtype: int64

In [10]:
rt_sched.calitp_itp_id.value_counts()

290    5896
300    1002
Name: calitp_itp_id, dtype: int64

## Charting

In [47]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

#import seaborn as sns

In [19]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
3821,290,SamTrans,1,FLX,FLX,2022-06-28,2021-04-16,2022-08-08,16,0,0.00,Tuesday,June


In [20]:
rt_sched>>count(_.month)

,month,n
0,June,3141
1,May,3757


In [21]:
rt_sched>>group_by(_.agency_name, _.month)>>summarize(avg = _.pct_w_vp.mean())

,agency_name,month,avg
0,Big Blue Bus,June,0.86
1,Big Blue Bus,May,0.84
2,SamTrans,June,0.74
3,SamTrans,May,0.86


In [22]:
#aggregating by date
day_pct = utils.agg_by_date(rt_sched, 'num_sched', 'num_vp')

In [23]:
day_pct

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,290,SamTrans,1,2022-05-01,Sunday,May,765,534,0.70
1,290,SamTrans,1,2022-05-02,Monday,May,1447,1043,0.72
2,290,SamTrans,1,2022-05-03,Tuesday,May,1447,1388,0.96
3,290,SamTrans,1,2022-05-04,Wednesday,May,1455,3,0.00
4,290,SamTrans,1,2022-05-05,Thursday,May,1445,1047,0.72
...,...,...,...,...,...,...,...,...,...
117,300,Big Blue Bus,0,2022-06-26,Sunday,June,770,498,0.65
118,300,Big Blue Bus,0,2022-06-27,Monday,June,1461,1013,0.69
119,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1461,1359,0.93
120,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1461,1367,0.94


In [24]:
#see if we have multiple date entries
day_pct>>count(_.service_date)>>arrange(-_.n)

,service_date,n
0,2022-05-01,2
1,2022-05-02,2
2,2022-05-03,2
3,2022-05-04,2
4,2022-05-05,2
...,...,...
56,2022-06-26,2
57,2022-06-27,2
58,2022-06-28,2
59,2022-06-29,2


## Percent by Day version 1

In [119]:
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==bbb)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [118]:
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'SamTrans Percent of Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [117]:
# if we want to change the color scale
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'SamTrans Percent of Scheduled Trips with RT Vehicle Position Data')).encode(color = alt.Color('pct_w_vp',
                       scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                       legend=alt.Legend(title=(utils.labeling('pct_w_vp')),
                                         symbolLimit=10))).properties(width=800)

alt.Chart(...)

## Percent by Day Version 2

In [ ]:
day_pct.sample()

In [ ]:
rt_sched.service_date.dtype

In [ ]:
day_pct.service_date.dtype

In [ ]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title=" Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(rt_sched>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%'))
)
(bar + rule).properties(width=900)

In [ ]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==st))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title="SamTrans Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(rt_sched>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp)', axis=alt.Axis(format='%'))
)

(bar + rule).properties(width=900)

In [ ]:
day_pct.sample()

## Percent by Day Version 3

In [ ]:
base = alt.Chart(day_pct).properties(width=550)
line = base.mark_line().encode(
    x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
    y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
    color='agency_name'
)

# # mean line
rule = alt.Chart(rt_sched>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%'))
)

line + rule

## Total Number of Scheduled Trips vs Vehicle Position Trips

In [29]:
df_long =  (day_pct>>select(_.agency_name,
                            _.calitp_itp_id,
                            _.service_date,
                         _.total_num_sched,
                         _.total_num_vp
                        ) 
             >> gather('measure',
                       'value',
                       _.total_num_sched,
                       _.total_num_vp
                      )
            )

In [116]:
(utils.bar_chart_over_time((df_long>>filter(_.calitp_itp_id==bbb)),
                           'service_date',
                           'value', 
                           'measure',
                           ',f', 
                           'x', 
                           'Big Blus Bus Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [115]:
(utils.bar_chart_over_time((df_long>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'value',
                           'measure', 
                           ',f',
                           'x', 
                           'SamTrans Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

## By Weekday

In [32]:
bbb_week = utils.groupby_onecol((day_pct>>filter(_.calitp_itp_id==bbb)), 'weekday', 'pct_w_vp')

In [33]:
bbb_week

,weekday,avg
1,Monday,0.73
5,Tuesday,0.88
6,Wednesday,0.91
4,Thursday,0.92
0,Friday,0.91
2,Saturday,0.70
3,Sunday,0.69


In [34]:
 cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [114]:
#using function- x axis is not sorted
utils.bar_chart_over_time(bbb_week, 
                          'weekday', 
                          'avg', 
                          'weekday', 
                          '%', 
                          cats_day,
                          'Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').mark_bar(size=50)

alt.Chart(...)

In [36]:
bbb_week_month = utils.groupby_twocol((day_pct>>filter(_.calitp_itp_id==bbb)), 'month', 'weekday', 'pct_w_vp', 'weekday')

In [38]:
# add column arg

In [113]:
utils.bar_chart_over_time(bbb_week_month,
                          'weekday', 
                          'avg',
                          'weekday',
                          '%', 
                          cats_day,
                          'Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').encode(column='month').mark_bar(size=50)

alt.Chart(...)

In [49]:
cats_month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [92]:
utils.bar_chart_over_time(bbb_week_month,
                          'avg',
                          'weekday',
                          'weekday',
                          '', 
                          cats_day, 
                          'Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').encode(x=alt.X('avg:Q', title=utils.labeling('avg'), axis=alt.Axis(format='%')),
                                                                                                                         row =alt.Row("month", sort=cats_month)).mark_bar(size=30)

alt.Chart(...)

In [98]:
day_pct.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
108,300,Big Blue Bus,0,2022-06-17,Friday,June,1409,1400,0.99


In [106]:
(utils.bar_chart_over_time(day_pct,
                           'service_date',
                           'pct_w_vp',
                           'agency_name', 
                           '%', 
                           cats_month,
                           "Average Percent of Scheduled Trips with RT Vehicle Position Data Over Time")).mark_line(size=3).properties(width=600)

alt.Chart(...)

In [107]:
(utils.bar_chart_over_time(bbb_week_month,
                           'weekday',
                           'avg',
                           'month',
                           '%',
                           cats_day,
                           "Average Percent of Scheduled Trips with RT Vehicle Position Data by Day")).mark_line(size=3).properties(width=600)

alt.Chart(...)

### Days with percents over 95% vehicle positions data present 

In [60]:
day_pct>>filter(_.pct_w_vp>.95)

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
2,290,SamTrans,1,2022-05-03,Tuesday,May,1447,1388,0.96
5,290,SamTrans,1,2022-05-06,Friday,May,1447,1396,0.96
11,290,SamTrans,1,2022-05-12,Thursday,May,1445,1389,0.96
12,290,SamTrans,1,2022-05-13,Friday,May,1447,1377,0.95
16,290,SamTrans,1,2022-05-17,Tuesday,May,1447,1390,0.96
17,290,SamTrans,1,2022-05-18,Wednesday,May,1455,1397,0.96
18,290,SamTrans,1,2022-05-19,Thursday,May,1445,1394,0.96
19,290,SamTrans,1,2022-05-20,Friday,May,1447,1395,0.96
26,290,SamTrans,1,2022-05-27,Friday,May,1447,1381,0.95
29,290,SamTrans,1,2022-05-30,Monday,May,765,732,0.96


## Group By Month

In [62]:
avg_month = utils.groupby_twocol(day_pct, 'agency_name', 'month', 'pct_w_vp', 'month')

In [110]:
(utils.bar_chart_over_time(avg_month, 
                           'month', 
                           'avg', 
                           'agency_name',
                           '%', 
                           cats_month, 
                           "")).encode(column='agency_name').mark_bar(size=150)

alt.Chart(...)

In [111]:
agg_weekday = utils.groupby_twocol(day_pct, 
                                   'agency_name',
                                   'weekday',
                                   'pct_w_vp', 
                                   'weekday')

In [112]:
(utils.bar_chart_over_time(agg_weekday,
                           'weekday', 
                           'avg',
                           'agency_name',
                           '%', 
                           cats_month,
                           "")).encode(column='agency_name').mark_bar(size=50)

alt.Chart(...)

## Using Functions (without DBT tables)

In [ ]:
#gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [ ]:
#gtfs_daily

In [ ]:
#gtfs_daily.service_date.min()

In [ ]:
#gtfs_daily.service_date.max()

In [ ]:
#rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

In [ ]:
#rt

In [ ]:
#rt['str_len'] = rt.trip_id.str.len()

In [ ]:
#rt.str_len.value_counts()

In [ ]:
#len(rt)

In [ ]:
#rt.trip_id.nunique()

In [ ]:
#len(gtfs_daily)

In [ ]:
#gtfs_daily.trip_id.nunique()

In [ ]:
#date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [ ]:

# from datetime import date, timedelta
# start_date = date(2022, 5, 1)
# end_date = date(2022, 5, 31)

In [ ]:
# daterange = pd.date_range(start_date, end_date)

In [ ]:
# (daterange)

In [ ]:
#above range does not work with function

In [ ]:
# dates = set()

# def daterange2(start, end):
#     for n in range(int((end - start).days) + 1):
#         yield start + timedelta(n)

# for single_date in daterange2(start_date, end_date):
#     dates.add(single_date.strftime('%Y-%m-%d'))

In [ ]:
# (dates)

In [ ]:
# rt['date'] = pd.to_datetime(rt['date'])

In [ ]:
# gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [ ]:
# single_date2 = '2022-05-08'

# (rt>>filter(_.date == single_date2))


In [ ]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [ ]:
# may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [ ]:
# may